In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

In [ ]:
listings = spark.read.csv("../data/listings.csv.gz", 
    header=True,
    inferSchema=True,
    sep=",", 
    quote='"',
    escape='"', 
    multiLine=True,
    mode="PERMISSIVE" 
)

In [ ]:
listings.printSchema()

In [ ]:
# 1. Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL


In [ ]:
# 2. Get number of properties that get more than 10 reviews per month


In [ ]:
# 3. Get a property that has more bathrooms than bedrooms


In [ ]:
# 4. Get 10 properties where the price is greater than 5,000. Collect the result as a Python list
# Remember to convert a price into a number first!


In [ ]:
# 5. Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5
# Consider using the "&" operator


In [ ]:
# 6. Get a list of properties with the following characteristics:
# * price < 150 OR more than one bathroom
# Use the "|" operator to implement the OR operator


In [ ]:
# 7. Get the highest listing price in this dataset
# Consider using the "max" function from "pyspark.sql.functions"


In [ ]:
# 8. Get the name and a price of property with the highest number of reviews per month
# Try to use "collect" method to get the price first, and then use it in a "filter" call 


In [ ]:
# 9. Get the number of hosts in the dataset


In [ ]:
# 10. Get listings with a first review in 2024
# Consider using the "year" function from "pyspark.sql.functions"
